In [1]:
import chess

import ipywidgets as widgets
import chess.pgn
import numpy as np
import io
pgn='''[Event "Biel Chess Festival"]
[Site "Biel SUI"]
[Date "2007.08.02"]
[Round "9"]
[White "Magnus Carlsen"]
[Black "Teimour Radjabov"]
[Result "1-0"]
[EventDate "?"]
[ECO "B07"]
[WhiteElo "2710"]
[BlackElo "2747"]
[PlyCount "55"]

1. e4 d6 2. d4 Nf6 3. Nc3 e5 4. Nge2 Nbd7 5. g3 c6 6. Bg2 b5 7. a3 Be7 8. O-O
O-O 9. h3 a5 10. g4 Ba6 11. Ng3 b4 12. Nce2 bxa3 13. Rxa3 d5 14. Re3 dxe4 15.
Re1 Qc7 16. Nf5 Bd8 17. g5 Nd5 18. Rxe4 f6 19. Neg3 g6 20. Nh6+ Kg7 21. dxe5
fxg5 22. e6 Kxh6 23. e7 Qb6 24. exf8=Q+ Nxf8 25. c4 Nf4 26. Qd6 Kg7 27. Bxf4
gxf4 28. Re7+ 1-0'''

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple, Combobox, HBox, VBox
import ipywidgets as widgets
#b = Button(description='HideShow')
import chess.engine

def get_score(b,white):
    cc= engine.analyse(b, chess.engine.Limit(depth=10))['score']
    
    if type(cc.relative) is chess.engine.Mate or  type(cc.relative) is chess.engine.MateGiven:
        if cc.relative< chess.engine.Cp(-500):
            return -5000
        else:
            return 5000
    
    
    if cc.relative.score() is None:
        print(cc)
        print(cc.score())
        breakpoint()
    return cc.relative.score() * (1 if white else (-1))
    
#Box( current_board
from stockfish import Stockfish
#stockfish = Stockfish(path=r'c:/gitproj/stockfish/stockfish-windows-x86-64-avx2.exe',depth=10)
engine = chess.engine.SimpleEngine.popen_uci(r'c:/gitproj/stockfish/stockfish-windows-x86-64-avx2.exe')
SCORECUTOFF=200
class Levels:
    pass 
from collections import defaultdict 
def calc_b(cur,white,oldeval,depth=2):
    lev= defaultdict(int)
    ll=list( calc_b_int(cur.board(),white,1,depth,oldeval,lev))
    return ll,lev
    
def calc_b_int(cur,white,curdep,maxdepth,oldeval,l):
    
    for z in cur.generate_legal_moves():
        cur.push(z)
        ev=get_score(cur,not white)
        g=ev-oldeval
        if white and g<(-1)*SCORECUTOFF: #too bad
            cur.pop()
            continue 
        elif g>SCORECUTOFF:
            cur.pop()
            continue
        
        l[curdep]+=1
        
        if curdep==maxdepth:
            yield ev
        else:
            yield from calc_b_int(cur,not white,curdep+1,maxdepth,ev,l)
        cur.pop()
        #stockfish.set_fen_position(curb.fen())
        
        #yield stockfish.get_evaluation()['value']
def display_game(pgn):      
    def rev(x):
        a.value=(a.value-1)
    def adv(x):
        a.value=(a.value+1)
    p=chess.pgn.read_game(io.StringIO(pgn))
    gam=p.game()
    current_board= widgets.HTML( str(gam.board()._repr_svg_())) 
    from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
    
    butl=Layout(width='35px',justify_content='center')
    ba=widgets.Button(description='<',layout=butl)
    ba.on_click(rev)
    ne=widgets.Button(description='>',layout=butl)
    ne.on_click(adv)
    mmax=len(list(gam.mainline_moves()))
    a = widgets.IntSlider(value=0, min=0, max=mmax)
    items = [ba,ne, widgets.Button(description='R',layout=butl),a]
    

    gb2 = widgets.HBox(items,layout=Layout(display='flex',
            flex_flow='row',
            align_items='stretch',
            width='100%'))
    def f2(a):
        cur=gam
        for k in range(a):
            cur=cur.next()
        display( widgets.HTML(str(cur.board()._repr_svg_())))
    
                
    def f3(a):
        cur=gam
        for k in range(a):
            cur=cur.next()
        
        iswhite= (a%2 ==0)
        
        curb=cur.board()
        init=get_score(curb,iswhite)
        #display(list(curb.generate_legal_moves()))
        try:
           
            o,lev=calc_b(cur,iswhite,init)
            #o.sort(reverse=iswhite)
            g=np.array(o)
            #if g is None:
            #    zz=None
            #else:
            #    g=g[g>-300] if iswhite else g[g<300]
            zz=np.std(g)
            display((init/100,zz/100,len(g),max(g)/100,min(g)/100))
            display(dict(lev))
        except Exception as e:
            import traceback
            print(traceback.format_exc())
            
        
        
        
    
    return VBox([gb2,widgets.interactive_output(f2,{'a':a}),widgets.interactive_output(f3,{'a':a})],layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='90%'
    ) )
    
p=chess.pgn.read_game(io.StringIO(pgn))
gg=p.game()
#list(calc_b(gg.next().next().next(),False,-55))


In [1]:
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
ThreadPool

In [2]:
mp.ThreadPool

AttributeError: module 'multiprocessing' has no attribute 'ThreadPool'

In [6]:
list(calc_b(gg.next().next().next(),False,-55))

[[42,
  124,
  101,
  95,
  121,
  93,
  122,
  21,
  15,
  72,
  84,
  101,
  89,
  85,
  82,
  92,
  116,
  72,
  97,
  125,
  79,
  79,
  79,
  132,
  85,
  66,
  88,
  85,
  109,
  84,
  -73,
  50,
  86,
  53,
  113,
  -116,
  -63,
  -112,
  21,
  -85,
  44,
  -92,
  -6,
  38,
  -7,
  -126,
  -110,
  -66,
  -66,
  -106,
  63,
  -105,
  14,
  -113,
  -95,
  -67,
  -83,
  35,
  -82,
  -104,
  -88,
  -87,
  -119,
  -109,
  -70,
  -110,
  -78,
  66,
  122,
  83,
  66,
  94,
  102,
  0,
  0,
  22,
  73,
  78,
  76,
  78,
  70,
  105,
  108,
  89,
  107,
  59,
  68,
  59,
  107,
  81,
  82,
  75,
  91,
  84,
  89,
  -13,
  91,
  91,
  44,
  89,
  7,
  95,
  67,
  93,
  67,
  97,
  -31,
  -40,
  -17,
  39,
  66,
  74,
  36,
  74,
  92,
  86,
  56,
  86,
  20,
  73,
  28,
  44,
  89,
  64,
  59,
  84,
  56,
  58,
  56,
  12,
  61,
  49,
  15,
  61,
  10,
  110,
  96,
  88,
  106,
  109,
  24,
  32,
  50,
  68,
  73,
  63,
  70,
  84,
  101,
  110,
  72,
  122,
  67,
  76,
  -60,
  66,
  71

In [4]:
display_game(pgn)

In [7]:
#engine.analyse(gg.next().board(), chess.engine.Limit(depth=10))['score'].white()

[]

In [51]:
engine.analyse(gg.next().next().next().board(), chess.engine.Limit(depth=10))['score']

PovScore(Cp(-55), BLACK)

In [ ]:
import numpy as np

In [ ]:
 a = widgets.FloatSlider(value=5, min=0, max=10,step=0.5,readout_format='.1f')

In [ ]:
dd=list(gg.board().generate_legal_moves())[0]

In [ ]:
z=gg.board().push_uci('g1h3')

In [ ]:
b=gg.board()

In [ ]:
b.push(dd)

In [ ]:
b

In [ ]:
gg.board()

In [ ]:
gg.board().push

In [ ]:
dd.uci()

In [ ]:
v= np.array([ 1,1])
w= np.array([ 0.96,-1.5])
print(v-w)
print(np.linalg.norm(v-w,0.5))
print(np.linalg.norm(v-w,1))
print(np.linalg.norm(v-w,2))

In [ ]:
w[w>0]

In [ ]:
w

In [ ]:
a = widgets.FloatSlider

In [ ]:
a

In [ ]:
widgets.IntSlider?

In [ ]:
a.value=7

In [ ]:
a

In [ ]:
from wxyz.svg import SVGBox


In [ ]:
io.StringIO?

In [ ]:
SVGBox(svg_file=r'c:\temp\tt.svg')

In [ ]:
open(r'c:\temp\tt.svg','wt').write(str(bb._repr_svg_()))

In [ ]:
bb=gg.board()

In [ ]:
gg.next().next().board().fen()

In [ ]:


display_game(pgn)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

a = widgets.IntSlider(value=5, min=0, max=10)

def f1(a):
    display(a)
    
def f2(a):
    display(a * 2)
    
out1 = widgets.interactive_output(f1, {'a': a})
out2 = widgets.interactive_output(f2, {'a': a})

display(a)
display(out1)
display(out2)